# Basic Usage of uvlf-hod Package

This notebook demonstrates the basic functionality of the `uvlf-hod` package:
1. Computing UV luminosity functions
2. Calculating galaxy bias
3. Exploring the UV-Halo Mass Relation
4. Parameter exploration

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from uvlf_hod import compute_luminosity_function, compute_galaxy_bias
from uvlf_hod.core import UVHaloMassRelation

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## Example 1: UV Luminosity Function at z=6

We'll compute the UV luminosity function using a standard set of parameters.

In [ ]:
# Define redshift
z = 6.0

# Define UV magnitude range
MUV = np.linspace(-22, -16, 25)

# Set up UVHMR parameters [eps0, log10(Mc), a, b]
uvhmr_params = [0.1, 11.5, 0.6, 0.35]

# Satellite parameters [log10(Mcut), log10(Msat), asat]
nsat_params = [10.0, 12.5, 1.0]

# UV magnitude scatter
sigma_UV = 0.35

print(f"Computing luminosity function at z={z}...")
print(f"UVHMR parameters: eps0={uvhmr_params[0]}, log(Mc)={uvhmr_params[1]}, a={uvhmr_params[2]}, b={uvhmr_params[3]}")
print(f"Satellite parameters: log(Mcut)={nsat_params[0]}, log(Msat)={nsat_params[1]}, asat={nsat_params[2]}")

In [ ]:
# Compute luminosity function
phi = compute_luminosity_function(
    MUV, z, uvhmr_params, nsat_params, sigma_UV
)

print(f"✓ Computation complete!")
print(f"  Magnitude range: {MUV.min():.1f} to {MUV.max():.1f}")
print(f"  Φ range: {phi.min():.2e} to {phi.max():.2e} Mpc^-3 mag^-1")

In [ ]:
# Plot the luminosity function
fig, ax = plt.subplots(figsize=(10, 7))

ax.semilogy(MUV, phi, 'b-', linewidth=2.5, label=f'z={z}')
ax.set_xlabel('$M_{\\mathrm{UV}}$', fontsize=16)
ax.set_ylabel('$\\Phi$ [Mpc$^{-3}$ mag$^{-1}$]', fontsize=16)
ax.set_title('UV Luminosity Function', fontsize=18, fontweight='bold')
ax.legend(fontsize=14)
ax.grid(True, alpha=0.3)
ax.tick_params(labelsize=12)

plt.tight_layout()
plt.savefig('luminosity_function.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Saved: luminosity_function.png")

## Example 2: Galaxy Bias

Calculate how galaxy clustering bias varies with UV magnitude.

In [ ]:
print("Computing galaxy bias...")

# Compute bias for same magnitude bins
bias = compute_galaxy_bias(MUV, z, uvhmr_params, nsat_params, sigma_UV)

print(f"✓ Computation complete!")
print(f"  Bias range: {bias.min():.2f} to {bias.max():.2f}")

In [ ]:
# Plot galaxy bias
fig, ax = plt.subplots(figsize=(10, 7))

ax.plot(MUV, bias, 'r-', linewidth=2.5, label=f'z={z}')
ax.set_xlabel('$M_{\\mathrm{UV}}$', fontsize=16)
ax.set_ylabel('Galaxy Bias $b_g$', fontsize=16)
ax.set_title('Galaxy Clustering Bias', fontsize=18, fontweight='bold')
ax.legend(fontsize=14)
ax.grid(True, alpha=0.3)
ax.tick_params(labelsize=12)

plt.tight_layout()
plt.savefig('galaxy_bias.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Saved: galaxy_bias.png")

## Example 3: UV-Halo Mass Relation

Explore the relationship between halo mass and UV magnitude.

In [ ]:
print("Exploring UV-Halo Mass Relation...")

# Create UVHMR instance
eps0, log_Mc, a, b = uvhmr_params
uvhmr = UVHaloMassRelation(
    z=z, eps0=eps0, Mc=10**log_Mc, a=a, b=b, add_dust=True
)

# Halo mass range
Mh_array = np.logspace(9, 13, 100)

# Get UV magnitudes
MUV_from_Mh = uvhmr.MUV(Mh_array)

print(f"✓ Relation computed for {len(Mh_array)} halo masses")

In [ ]:
# Plot UV-Halo Mass Relation
fig, ax = plt.subplots(figsize=(10, 7))

ax.plot(np.log10(Mh_array), MUV_from_Mh, 'g-', linewidth=2.5)
ax.set_xlabel('$\\log_{10}(M_h / M_\\odot)$', fontsize=16)
ax.set_ylabel('$M_{\\mathrm{UV}}$', fontsize=16)
ax.set_title('UV-Halo Mass Relation', fontsize=18, fontweight='bold')
ax.invert_yaxis()  # Brighter magnitudes on top
ax.grid(True, alpha=0.3)
ax.tick_params(labelsize=12)

plt.tight_layout()
plt.savefig('uvhmr.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Saved: uvhmr.png")

## Example 4: Key Results Summary

Let's examine some specific values from our calculations.

In [ ]:
# Find characteristic values
idx_bright = np.argmin(np.abs(MUV + 20))

print("="*60)
print("Key Results:")
print("="*60)
print(f"\nAt M_UV = {MUV[idx_bright]:.1f}:")
print(f"  Φ = {phi[idx_bright]:.2e} Mpc^-3 mag^-1")
print(f"  Bias = {bias[idx_bright]:.2f}")

# Halo mass for bright galaxy
Mh_bright = uvhmr.Mhalo(MUV[idx_bright])
print(f"  Typical halo mass = {Mh_bright:.2e} M_sun")

# SFR
sfr_bright = uvhmr.sfr(Mh_bright)
print(f"  Star formation rate = {sfr_bright:.1f} M_sun/yr")

print("\n" + "="*60)

## Example 5: Parameter Exploration

Let's see how varying the star formation efficiency (eps0) affects the luminosity function.

In [ ]:
print("Exploring parameter dependence...")

# Explore effect of varying eps0
eps0_values = [0.05, 0.1, 0.15, 0.2]
colors = plt.cm.viridis(np.linspace(0, 1, len(eps0_values)))

fig, ax = plt.subplots(figsize=(10, 7))

for eps0, color in zip(eps0_values, colors):
    uvhmr_params_var = [eps0, 11.5, 0.6, 0.35]
    phi_var = compute_luminosity_function(
        MUV, z, uvhmr_params_var, nsat_params, sigma_UV
    )
    ax.semilogy(MUV, phi_var, linewidth=2.5, color=color, 
                label=f'$\\epsilon_0={eps0}$')

ax.set_xlabel('$M_{\\mathrm{UV}}$', fontsize=16)
ax.set_ylabel('$\\Phi$ [Mpc$^{-3}$ mag$^{-1}$]', fontsize=16)
ax.set_title('Effect of Star Formation Efficiency', fontsize=18, fontweight='bold')
ax.legend(fontsize=14)
ax.grid(True, alpha=0.3)
ax.tick_params(labelsize=12)

plt.tight_layout()
plt.savefig('parameter_exploration.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Saved: parameter_exploration.png")

## Example 6: Multi-Redshift Analysis

Compare luminosity functions at different redshifts.

In [ ]:
print("Computing multi-redshift luminosity functions...")

redshifts = [5, 6, 7, 8]
colors = plt.cm.plasma(np.linspace(0, 0.8, len(redshifts)))

fig, ax = plt.subplots(figsize=(10, 7))

for z_val, color in zip(redshifts, colors):
    phi_z = compute_luminosity_function(
        MUV, z_val, uvhmr_params, nsat_params, sigma_UV
    )
    ax.semilogy(MUV, phi_z, linewidth=2.5, color=color, label=f'z={z_val}')

ax.set_xlabel('$M_{\\mathrm{UV}}$', fontsize=16)
ax.set_ylabel('$\\Phi$ [Mpc$^{-3}$ mag$^{-1}$]', fontsize=16)
ax.set_title('Redshift Evolution of UV Luminosity Function', 
             fontsize=18, fontweight='bold')
ax.legend(fontsize=14)
ax.grid(True, alpha=0.3)
ax.tick_params(labelsize=12)

plt.tight_layout()
plt.savefig('multi_redshift.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Saved: multi_redshift.png")

## Summary

This notebook demonstrated:
- ✅ Computing UV luminosity functions
- ✅ Calculating galaxy bias
- ✅ Exploring the UV-halo mass relation
- ✅ Parameter exploration
- ✅ Multi-redshift analysis

All figures have been saved to the current directory.

For more examples and documentation, see:
- `GETTING_STARTED.md`
- Full API documentation
- Additional example notebooks

In [ ]:
print("\n" + "="*60)
print("Example completed successfully!")
print("="*60)
print("\nGenerated files:")
print("  - luminosity_function.png")
print("  - galaxy_bias.png")
print("  - uvhmr.png")
print("  - parameter_exploration.png")
print("  - multi_redshift.png")